## Figure 3
### Calculate steady state phonon number for the three-level system as a function of decay rates: 
### $\gamma$ vs $\gamma_2$ and $\gamma_1$ vs $\gamma_2$

In [ ]:
# Import Libraries
import Phonon_Number as pn
import parameters as params

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import clear_output
from scipy.io import savemat, loadmat

### (a) $\gamma$ vs $\gamma_2$

In [ ]:
'''
Import and define parameters
'''
# Import parameters
omega_21, temperature, g, pump, T2, T1, gamma = params.three_level_system_params()

N = 50

data_points = 50                             # number of data points in arrays
T2 = np.logspace(-2, 2, data_points)*g       # in GHz
gamma = np.logspace(-6, -2, data_points)*g   # in GHz

dephasing = 0                                # in GHz

# initializing arrays
phonon_gamma_gamma2 = np.zeros((len(T2), len(gamma)))

In [ ]:
'''
Iterating over to calculate simulation values
'''

for i in range(len(T2)):
    for j in range(len(gamma)):
        
        # Using simulations
        phonon_gamma_gamma2[i,j] = pn.QD_simulation(g, pump, T2[i], T1, gamma[j], dephasing, nth, N)
        
        # progress monitor
        if(np.mod(i,10) == 0 and np.mod(j,10) == 0):
            print((i/100,j/100))
            
clear_output()
print('Calculations done!')

In [ ]:
'''
Save data into .mat file
'''

decay_rates_a = {'data_points' : data_points,
                 'omega' : omega,
                 'temperature' : temp,
                 'g' : g,
                 'pump' : pump,
                 'T2' : T2,
                 'T1' : T1,
                 'gamma' : gamma,
                 'dephasing': dephasing,
                 'N' : N,
                 'phonon_gamma_gamma2' : phonon_gamma_gamma2,
                 }

savemat("./data files/decay_rates_a.mat", decay_rates_a) # saving data

### (b) $\gamma_1$ vs $\gamma_2$

In [ ]:
'''
Import and define parameters
'''

# Import parameters
omega_21, temperature, g, pump, T2, T1, gamma = params.three_level_system_params()

data_points = 50                             # number of data points
T2 = np.logspace(-2, 2, data_points)*g       # in GHz
T1 = np.logspace(-2, 2, data_points)*g       # in GHz

gamma = 10**-4*g                             # in GHz
dephasing = 0                                # in GHz

# initializing arrays
phonon_gamma1_gamma2 = np.zeros((len(T1), len(T2)))

In [ ]:
'''
Iterating over to calculate simulation values
'''

for i in range(len(T1)):
    for j in range(len(T2)):
        
        # Using simulations
        phonon_gamma1_gamma2[i,j] = coherent_simulation(g, pump, T2[j], T1[i], gamma, dephasing, nth, N)
        
        # Progress monitor
        if(np.mod(i,10) == 0 and np.mod(j,10) == 0):
            print((i/100,j/100))

clear_output()
print('Calculations done!')

In [ ]:
'''
Save data into .mat files
'''

decay_rates_b = {'data_points' : data_points,
                 'omega' : omega,
                 'temperature' : temp,
                 'g' : g,
                 'pump' : pump,
                 'T2' : T2,
                 'T1' : T1,
                 'gamma' : gamma,
                 'dephasing': dephasing,
                 'N' : N,
                 'phonon_gamma1_gamma2' : phonon_gamma1_gamma2,
                 }

savemat("./data files/decay_rates_b.mat", decay_rates_b) # saving data

In [ ]:
'''
Import data files
'''

# import data files and define variables
decay_rates_a = loadmat("./data files/decay_rates_a.mat")
decay_rates_b = loadmat("./data files/decay_rates_b.mat")

g = int(decay_rates_a['g'])

phonon_gamma_gamma2 = decay_rates_a['phonon_gamma_gamma2']
phonon_gamma1_gamma2 = decay_rates_b['phonon_gamma1_gamma2']

In [ ]:
'''
Plotting
'''

# plot specifications
fig = plt.figure(constrained_layout=True)
spec = fig.add_gridspec(ncols=2, nrows=1)  # subplot grid
locator = ticker.LogLocator(base=10)

'''
Figure 3(a): gamma vs T2
'''
# Import data
gamma = decay_rates_a['gamma']
T2 = decay_rates_a['T2']

[X,Y] = np.meshgrid(T2/g, gamma/g) # define X,Y for contour plot
fig.add_subplot(spec[0, 0])
levels = np.logspace(-6,0,20) # finetune gradation in contour plot

plt.contourf(X,Y, phonon_gamma_gamma2.T, locator=ticker.LogLocator(), levels = levels, cmap = 'viridis_r') # contour plot
cbar = plt.colorbar(ticks=[10**-6, 10**-4, 10**-2, 1], aspect = 20, shrink = 0.7)

plt.xscale('log')
plt.yscale('log')
plt.ylabel('$\gamma/g$')
plt.xlabel('$\gamma_2 /g$')
plt.title('(a)')
plt.xticks([0.01, 1, 100])
plt.yticks([0.000001, 0.0001, 0.01])
ax = plt.gca()
ax.set(aspect='equal')

'''
Figure 3(b): T1 vs T2
'''

T2 = decay_rates_b['T2']
T1 = decay_rates_b['T1']

[X,Y] = np.meshgrid(T2/g, T1/g) # define X,Y for contour plot
levels = np.logspace(-4,0,20) # finetune gradation in contour plot
fig.add_subplot(spec[0, 1])

plt.contourf(X,Y, phonon_gamma1_gamma2, locator=ticker.LogLocator(), levels = levels, cmap = 'viridis_r') # contour plot

# horizontal line (to change)
plt.axhline(y = pump/g, color = 'black', linestyle = '--', linewidth = 4) 
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$\gamma_2/g$')
plt.ylabel('$\gamma_1/g$')
plt.title('(b)')
plt.yticks([0.01, 1, 100])
ax = plt.gca()
ax.set(aspect='equal')
cbar = plt.colorbar(ticks=locator, aspect = 20, shrink = 0.7)
plt.rcParams.update({'font.size': 40})
cbar.set_label(r'$\mathcal{F} = \langle b^{\dagger}b \rangle_s/n_{th}$')

fig = plt.gcf()
fig.set_size_inches((23, 9))

fig.savefig('./figures/decay_rates.pdf')
plt.show()